In [ ]:
# ===============================================================
#   Non-coherent, amplitude-based TEMPEST image reconstruction
# ===============================================================

import numpy as np, imageio, h5py
from scipy.signal import resample_poly, firwin, lfilter

# ---------- Parameters ----------
Fs = 60_000_000.0
pixel_rate = 148_500_000.0
h_total, v_total = 2200, 1125
up, down = 99, 40
BW = 16e6
numtaps = 129
file_in = './signals/bb_20260206_103810_CF_445.500MHz_FS_60.000MSPS_T0.5s_0.1s.mat'
out_prefix = 'p1'

# ---------- File Reading ----------
def read_bb_or_mat(fname):
    if fname.endswith('.mat'):
        with h5py.File(fname, 'r') as f:
            keys = list(f.keys())
            if 'x' in keys:
                d = np.array(f['x'])
                # ---- Case A: Standard complex ----
                if np.iscomplexobj(d):
                    return d.squeeze().astype(np.complex64)
                # ---- Case B: Structured dtype {real, imag} ----
                if isinstance(d.dtype, np.dtype) and {'real','imag'}.issubset(d.dtype.fields.keys()):
                    real = d['real']
                    imag = d['imag']
                    return (real + 1j*imag).astype(np.complex64).squeeze()
                # ---- Case C: Nx2 or 2xN real matrix ----
                if d.ndim == 2 and d.shape[1] == 2:
                    return (d[:,0] + 1j*d[:,1]).astype(np.complex64)
                if d.ndim == 2 and d.shape[0] == 2:
                    return (d[0,:] + 1j*d[1,:]).astype(np.complex64)
                # ---- Case D: Single row/column vector ----
                if d.ndim == 2 and (d.shape[0] == 1 or d.shape[1] == 1):
                    return d.flatten().astype(np.complex64)
                raise ValueError(f"Unsupported dataset shape or dtype for 'x': shape={d.shape}, dtype={d.dtype}")
            elif '/BasebandData/IQData' in f:
                iq = np.array(f['/BasebandData/IQData'])
                iq = iq[:,0] + 1j*iq[:,1]
                return iq.astype(np.complex64)
            else:
                raise ValueError(f"Cannot find dataset 'x' or '/BasebandData/IQData'. Top keys: {keys}")
    elif fname.endswith('.bb'):
        with h5py.File(fname, 'r') as f:
            iq = np.array(f['/BasebandData/IQData'])
            iq = iq[:,0] + 1j*iq[:,1]
        return iq.astype(np.complex64)
    else:
        raise ValueError('Unsupported file extension')


rx = read_bb_or_mat(file_in)
print(f'Loaded {len(rx)/1e6:.2f} M samples.')

# ---------- DC Removal ----------
rx -= np.mean(rx[:min(len(rx), 10000)])

# ---------- Frequency Offset Estimation ----------
def estimate_freq_offset(x, Fs, m=2000):
    r = x[m:] * np.conj(x[:-m])
    ang = np.angle(np.sum(r))
    return ang / (2*np.pi*m/Fs)

foff = estimate_freq_offset(rx, Fs)
print(f'Estimated freq offset: {foff:.1f} Hz')
n = np.arange(len(rx))
rx *= np.exp(-2j*np.pi*foff*n/Fs)

# ---------- Bandpass Filtering ----------
nyq = Fs/2
b = firwin(numtaps, cutoff=BW/nyq)
rx_filt = lfilter(b, 1.0, rx)

# ---------- Resampling to Pixel Rate ----------
rx_pix = resample_poly(rx_filt, up, down)
print(f'Resampled to {len(rx_pix)/1e6:.2f} M samples at {pixel_rate/1e6:.1f} MHz')

# ---------- Reshape to One Frame ----------
Npix = h_total * v_total
if len(rx_pix) >= Npix:
    img_complex = rx_pix[:Npix].reshape((v_total, h_total))
else:
    raise ValueError('Samples not enough for one frame')

# ---------- Save Complex IQ Mapping ----------
np.save(out_prefix + '.npy', img_complex)
I, Q = np.real(img_complex), np.imag(img_complex)
vmin, vmax = np.percentile(np.hstack((I.ravel(), Q.ravel())), (1, 99))
Ir = np.clip((I - vmin) / (vmax - vmin), 0, 1)
Qr = np.clip((Q - vmin) / (vmax - vmin), 0, 1)
vis = np.stack([Ir, Qr, np.zeros_like(Ir)], axis=-1)
imageio.imwrite(out_prefix + '.png', (vis * 255).astype(np.uint8))
print('Saved:', out_prefix + '.npy / .png')

# ---------- Amplitude Image (Raw Linear) ----------
amp = np.abs(img_complex)
vmin, vmax = np.percentile(amp, (1, 99))
amp_norm = np.clip((amp - vmin) / (vmax - vmin), 0, 1)
imageio.imwrite(out_prefix + '_amp.png', (amp_norm * 255).astype(np.uint8))

# ---------- Amplitude Image + Linear Gain ----------
GAIN = 3
amp_boost = np.clip(amp_norm * GAIN, 0, 1)
imageio.imwrite(out_prefix + '_amp_gain.png', (amp_boost * 255).astype(np.uint8))

# ---------- Amplitude Image + Log Compression + Gain ----------
amp_db = 20 * np.log10(amp + 1e-8)
vmin, vmax = np.percentile(amp_db, (1, 99))
amp_db_norm = np.clip((amp_db - vmin) / (vmax - vmin), 0, 1)
GAIN_LOG = 1.4
amp_db_boost = np.clip(amp_db_norm * GAIN_LOG, 0, 1)
imageio.imwrite(out_prefix + '_amp_log_gain.png', (amp_db_boost * 255).astype(np.uint8))

print('Saved amplitude variants: _amp.png, _amp_gain.png, _amp_log_gain.png')


C:\UserData\AppData\Local\Temp\ipykernel_24900\1873354107.py:24: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  d = np.array(f['x'])


Loaded 6.00 M samples.
Estimated freq offset: -7001.8 Hz
Resampled to 14.85 M samples at 148.5 MHz
Saved: image6_35db_v2.npy / .png
Saved amplitude variants: _amp.png, _amp_gain.png, _amp_log_gain.png
